In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform=ToTensor(),
    download=True
)

test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform=ToTensor(),
    download=True
)

In [ ]:
train_data.data.size()
test_data.data.size()

In [ ]:
train_data.targets.size()

In [ ]:
from torch.utils.data import DataLoader

loaders = {
    'train': DataLoader(train_data, batch_size=64, shuffle=True, num_workers=1),
    'test': DataLoader(test_data, batch_size=64, shuffle=True, num_workers=1)
}

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))  
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1) 

In [ ]:
import torch 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loaders['train']):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(loaders['train'].dataset),
                100. * batch_idx / len(loaders['train']), loss.item()))

def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in loaders['test']:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(loaders['test'].dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(loaders['test'].dataset),
        100. * correct / len(loaders['test'].dataset)))
    

In [ ]:
for epoch in range(1, 11):
    train(epoch)
    test(epoch)

In [ ]:
device

In [ ]:
import os
import torch
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

def load_custom_image(image_path, transform=None):
    """
    Load the image and prepare it for model prediction.
    
    Args:
    - image_path (str): Path to the image file.
    - transform (transforms): Transformations to apply to the image.
    
    Returns:
    - tensor: Prepared image for prediction.
    """
    if transform is None:
        transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=1),  
            transforms.Resize((28, 28)),  
            transforms.ToTensor(),  
            transforms.Normalize((0.1307,), (0.3081,))  
        ])
    
    image = Image.open(image_path)
    image_tensor = transform(image)
    
    return image_tensor

def predict_image(model, image_path, device):
    """
    Predict the digit for the given image.
    
    Args:
    - model: Trained CNN model.
    - image_path (str): Path to the image file.
    - device: The device (cuda/cpu).
    
    Returns:
    - int: Predicted digit.
    - numpy.ndarray: Image to display.
    """
    model.eval()
    data = load_custom_image(image_path)
    data = data.unsqueeze(0).to(device)  
    
    with torch.no_grad():
        output = model(data)
        prediction = output.max(1, keepdim=True)[1]
    
    image = data.cpu().numpy().squeeze()
    
    return prediction.item(), image

def predict_images_in_folder(model, folder_path, device):
    """
     Predict digits for all images in a folder.
    
    Args:
    - model: Trained CNN model.
    - folder_path (str): Path to the folder containing images.
    - device: The device (cuda/cpu).
    """
    if not os.path.exists(folder_path):
        print(f"The folder {folder_path} does not exist!")
        return
    
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            image_path = os.path.join(folder_path, filename)
            try:
                prediction, image = predict_image(model, image_path, device)
                
                plt.figure(figsize=(4, 4))
                plt.imshow(image.squeeze(), cmap='gray')
                plt.title(f'Prediction: {prediction}')
                plt.axis('off')
                plt.show()
            
            except Exception as e:
                print(f"Error processing {filename}: {e}")

predict_images_in_folder(model, 'Images/MNIST_Digits/', device)